This script aims at generating input files to be used as inputs to the CMD. Starting from the ZINC lead-like clean compounds (clle.smi), it applies divsamp0 to create an orthogonal subset. It adds properties to the molecules and save as csv file (div0le.csv). In parallel, it creates a control csv file with the first 310K clustered molecules of the ZINC lead-like clean compounds (clusle.csv); this file is supposed to have much lower diversity since we are using the largest clusters from clle.smi. 

The same process is done but starting from the ZINC drug-like clean compounds (cldr.smi)

In [3]:
# Import myfunc at cix folder
%matplotlib inline
import matplotlib.pyplot as plt
import sys
sys.path.insert(0, '../cix')
import myfuncs as mf
import pandas as pd
from rdkit.Chem import rdBase, RDConfig
from rdkit import Chem
rdBase.DisableLog('rdApp.*') # To make rdkit silent
from rdkit.Chem import PandasTools as pt
from rdkit.Chem import Descriptors
import numpy as np
import chemfp
import csv
import time
import os

In [4]:
####################################################
### LEADS orthogonal file
####################################################

## First import ZINC SMILES file and generate the corresponding arena

start = time.time()

smidfl = mf.smisf2smidf("./clle.smi", noid = False, random = True, seed = 123)

arl = mf.smidf2arena(smidfl)

end = time.time()

eltime = end - start

print('Total time: ' + time.strftime("%H:%M:%S", time.gmtime(eltime)))

Total time: 01:23:18


In [5]:
## Apply divsamp0 to generate an orthogonal set of 310K molecules

sell = mf.divsamp0(arl, nlimit = 310000)

i=4591274; nsel=310000; nneig=3685062; Sampling time: 11:46:58


In [9]:
# Add ids and save as SMILES file

nids = [arl.ids[x] for x in sell]
smidfl[smidfl.id.isin(nids)].to_csv("div0le.smi", index = False, sep = " ", header = False)

In [4]:
# Create with these molecules a csv file with properties to be able to run CMD
# This is a similiar process to the one used in GenerateNP in exp3

data = pd.read_csv("./div0le.smi", header = None, names = ['smiles'])

# Create mol objects
data['mol'] = data['smiles'].apply(Chem.MolFromSmiles)

# Create column with non-isomeric SMILES (we'll use that)
data["smi"] = data['mol'].apply(Chem.MolToSmiles, isomericSmiles=False)

# Generate properties and add to dataframe
data['mw'] = data['mol'].apply(Descriptors.MolWt)
data['logp'] = data['mol'].apply(Descriptors.MolLogP)
data['qed'] = data['mol'].apply(Descriptors.qed)

# Delete column of mol objects
del data['mol']

# Save csv file with non-isomeric smiles and properties
data[["smi","mw","logp","qed"]].to_csv('./div0le.csv', index = False)

In [ ]:
####################################################
### LEADS clustered file
####################################################

# We generate a dataframe with smiles from the zinc lead-like clean group of molecules. 
smidfl = mf.smisf2smidf("./clle.smi", noid = False, seed = 123)

# We cluster the molecules
cls = mf.clusmidf(smidfl)

In [ ]:
### DRUGS file

start = time.time()

smidfd = mf.smisf2smidf("./cldr.smi", noid = False, random = True, seed = 123)

ard = mf.smidf2arena(smidfd)

end = time.time()

eltime = end - start

print('Total time: ' + time.strftime("%H:%M:%S", time.gmtime(eltime)))

In [ ]:
seld = mf.divsamp0(ard, nlimit = 310000)

In [ ]:
nids = [ard.ids[x] for x in seld]

In [ ]:
smidfd[smidfd.id.isin(nids)].to_csv("div0dr.smi", index = False, sep = " ", header = False)